In [ ]:
# !pip install openai
# !rm /work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai/credentials.json

In [4]:
import os
import csv
import time
import json
import pandas as pd
from tqdm import tqdm

from openai import OpenAI
from dotenv import load_dotenv

In [5]:
load_dotenv("credentials.env")
api_key = os.getenv("api_key")

In [6]:
client = OpenAI(api_key=api_key)
model = "gpt-4" # gpt-3.5-turbo-instruct gpt-4

## Get Data

In [4]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/astha/696DS-named-entity-extraction-and-linking-for-KG-construction/code/mc1/mc1_preprocess/"
file = "mc1_chunked_data.json"

with open(os.path.join(data_path, file), 'r') as f:
    data = json.load(f)

In [34]:
type(data), data.keys()

(dict, dict_keys(['Documents']))

In [35]:
data['Documents']['3']

[{'chunk_index': 0,
  'chunk_text': 'SOURCE: All News Today\n\nTITLE: RALLY SCHEDULED IN SUPPORT OF INCREASED GOVERNMENT ACCOUNTABILITY \n\n \n\nPUBLISHED: 2012/04/09\n \n\nLOCATION: ABILA, Kronos \n\nSilvia Marek, leader of the Protectors of Kronos social action organization is appealing to the citizens of Kronos to join her for a day of peaceful protest tomorrow.\n\nMarek asks all concerned citizens to join her to bring awareness to the problem of political corruption and favoritism pervasive in the offices of the government of Kronos.\n\n"I\'m calling for an end to the heinous corruption, the nepotism and the favoritism displayed by our elected officials"',
  'chunk_tokens': 105}]

In [ ]:
len(data['historical_document']['reports'][0])

### RElations

In [5]:
entities_data = pd.read_csv("openai_mc1_entities_Apr13_processed.csv")
# Data0 = Data0.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
#                             4: 'input_tokens',  5: 'output_tokens'})
print(entities_data.shape)
entities_data.head()

(1339, 7)


,doc_key,chunk_index,entities,latency,input_tokens,output_tokens,processed_entities
0,0,0,Kyrla Halford; University of Abila; Kronos; Oc...,4.019342,713,103,Kyrla Halford; University of Abila; Kronos; Oc...
1,0,1,Master’s Thesis; Paper SW1138; Esme Nesmith; E...,1.967966,705,38,Master’s Thesis; Paper SW1138; Esme Nesmith; E...
2,0,2,1997; Elodis; Kronos; Tiskele River; Protector...,1.920458,690,31,1997; Elodis; Kronos; Tiskele River; Protector...
3,0,3,1997; Elodis; Elodis Community Health Center; ...,3.157337,690,19,1997; Elodis; Elodis Community Health Center; ...
4,0,4,Initial Stage; Grassroots Effort; Council; Elo...,2.945101,733,70,Initial Stage; Grassroots Effort; Council; Elo...


In [6]:
entities_data.loc[lambda df: df.doc_key == 3]

,doc_key,chunk_index,entities,latency,input_tokens,output_tokens,processed_entities
32,3,0,All News Today; 2012/04/09; ABILA; Kronos; Sil...,1.462747,658,28,All News Today; 2012/04/09; ABILA; Kronos; Sil...


In [7]:
data_json = {}
for idx in range(entities_data.shape[0]):
    doc_key = entities_data["doc_key"][idx]
    chunk_index = entities_data["chunk_index"][idx]
    processed_entities = entities_data["processed_entities"][idx]
    
    if doc_key not in data_json:
        data_json[doc_key] = {}
        
    data_json[doc_key][chunk_index] = processed_entities

In [39]:
data_json[3]

{0: 'All News Today; 2012/04/09; ABILA; Kronos; Silvia Marek; Protectors of Kronos'}

## Get prompt

### Entities Extraction

In [ ]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file_v2 = "Re-DocRED_Processed_Train.csv"
    
def get_entity_example(idx, datatmp):
    allEntities = set()
    for item in datatmp["Triplets"][idx].split("\n"):
        item = item.split(" | ")
        if len(item) == 3:
            allEntities.add(item[0])
            allEntities.add(item[2])

    return datatmp["Text"][idx], '; '.join(list(allEntities))
    
def get_entities_prompt(text):
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exout1 = get_entity_example(1, data_v2)
    ex2, exout2 = get_entity_example(2, data_v2)

    prompt=f'''Task: Please detect all the entities from the given input Text.
Entities could be people, organization, places, concepts, dates or any other proper nouns present in the text. \
Use the following examples as reference to understand the task. \
Give the output in the same format as given in the Example Entities Output, i.e., separated by a semicolon, ';'.

Example Text 1: {ex1}
Example Entities Output 1: {exout1}

Example Text 2: {ex2}
Example Entities Output 2: {exout2}

Text: {text}
Entities Output:'''
    
    return prompt

def get_system_msg():
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exout1 = get_entity_example(1, data_v2)
    ex2, exout2 = get_entity_example(2, data_v2)

    prompt=f'''You are a helpful assistant and an expert in named entity extraction.
Task: Please detect all the entities from the given input Text.
Entities could be people, organization, places, concepts, dates or any other proper nouns present in the text. \
Use the following examples as reference to understand the task. \
Give the output in the same format as given in the Example Entities Output, i.e., separated by a semicolon, ';'.

Example Text 1: {ex1}
Example Entities Output 1: {exout1}

Example Text 2: {ex2}
Example Entities Output 2: {exout2}'''
    
    return prompt

def get_content(text):
    
    prompt=f'''Text: {text}
Entities Output:'''
    
    return prompt

In [ ]:
print(get_system_msg())
print(get_content("hey"))

### Relations Extraction

In [8]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file_v2 = "Re-DocRED_Processed_Train.csv"
    
def get_example(idx, datatmp):
    allEntities = set()
    for item in datatmp["Triplets"][idx].split("\n"):
        item = item.split(" | ")
        # print(item, len(item))
        if len(item) == 3:
            allEntities.add(item[0])
            allEntities.add(item[2])

    allEntities = list(allEntities)
    # print(allEntities)
    # return datatmp["Text"][idx], allEntities, datatmp["Triplets"][idx]
    return datatmp["Text"][idx], "; ".join(allEntities), datatmp["Triplets"][idx]
    
def get_relations_prompt(text, entities):
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exent1, exout1 = get_example(1, data_v2)
    ex2, exent2, exout2 = get_example(2, data_v2)

    prompt=f'''Task Description:
The task is to extract Relations between the Entity List for given text, in the form of triplets. \
Extract triplets from the given Text based solely on the relationships present in the text. \
Ensure that entities are chosen directly from the provided Entity List to maintain accuracy. \
Avoid duplicating triplets in the output. Use the provided Example Text and Relations Output as references \
to understand how to identify meaningful relationships between entities from Entity List. \
Pay attention to all potential relations between all the entities and include them in the output.

Example Text 1: {ex1}
Entity List of Text 1: {exent1}
Relations Output of Text 1: {exout1}

Example Text 2: {ex2}
Entity List of Text 2: {exent2}
Relations Output of Text 2: {exout2}

Text: {text}
Entity List: {entities}
Relations Output:'''
    
    return prompt

In [9]:
print(get_relations_prompt("text", "entities"))

Task Description:
The task is to extract Relations between the Entity List for given text, in the form of triplets. Extract triplets from the given Text based solely on the relationships present in the text. Ensure that entities are chosen directly from the provided Entity List to maintain accuracy. Avoid duplicating triplets in the output. Use the provided Example Text and Relations Output as references to understand how to identify meaningful relationships between entities from Entity List. Pay attention to all potential relations between all the entities and include them in the output.

Example Text 1: Udawalawe National Park lies on the boundary of Sabaragamuwa and Uva Provinces, in Sri Lanka. The national park was created to provide a sanctuary for wild animals displaced by the construction of the Udawalawe Reservoir on the Walawe River, as well as to protect the catchment of the reservoir. The reserve covers of land area and was established on 30 June 1972. Before the designation

In [ ]:
get_entities_prompt("text")

## Predictions

In [9]:
def get_prediction(text):
    
    prompt = get_entities_prompt(text)
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature = 0,
        max_tokens = 256,
    )
    time_diff = time.time() - start
    
    return response, time_diff

def get_chat_prediction(messages):
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = 0,
        max_tokens = 256,
    )
    time_diff = time.time() - start
    
    return response, time_diff


def get_relation_prediction(text, entities):
    
    prompt = get_relations_prompt(text, entities)
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    time_diff = time.time() - start
    
    return response, time_diff

In [13]:
output_data = []
output_filename = "openai_mc1_triplets_Apr13.csv"

### Entity Extraction

In [ ]:
debug = False

all_keys = list(data['Documents'].keys())

for i in tqdm(range(len(all_keys))):
    
    doc_key = all_keys[i]
    
    if doc_key != '5' and debug:
        continue
    
    # messages = [{"role": "system", "content": get_system_msg()}]
    
    for item in data['Documents'][i]:
        
        # print(messages)
    
        #
        text = item['chunk_text']
        chunk_idx = item['chunk_index']
        
        # messages.append({"role": "user", "content": get_content(text)})
        
        output, time_diff = get_prediction(text)
        # output, time_diff = get_chat_prediction(messages)
        # output, time_diff = get_relation_prediction(i)

        outputText = output.choices[0].message.content
        input_tokens = output.usage.prompt_tokens
        output_tokens = output.usage.completion_tokens
        
        # messages.append({"role": "assistant", "content": outputText})

        if debug:
            print(f"\nInput - {text}")
            # print(f"\nGT - {data['Triplets'][i]}")
            print(f"\noutput - {outputText}")

        output_data.append([doc_key, chunk_idx, outputText, time_diff,
                           input_tokens, output_tokens])

        if i % 10 == 0 and i > 1:

            with open(output_filename, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(output_data)
                
        # if debug: break
        
    if debug: break
    
    with open(output_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_data)

### Relation Extraction

In [14]:
processed = set()

In [16]:
processed

{('0', 0), ('0', 1), ('0', 2), ('0', 3), ('0', 4), ('0', 5)}

In [20]:
debug = False

all_keys = list(data['Documents'].keys())

for i in tqdm(range(len(all_keys))):
    
    doc_key = all_keys[i]
    
    if doc_key != '3' and debug:
        continue
        
    if doc_key != '400':
        continue
    
    # messages = [{"role": "system", "content": get_system_msg()}]
    
    for item in data['Documents'][doc_key]:
        
        # print(doc_key, item)
        
        # print(messages)
    
        #
        text = item['chunk_text']
        chunk_idx = item['chunk_index']
        
        # print(doc_key, chunk_idx)
        
        
        # if (doc_key, chunk_idx) != ('3',0):
        #     continue
            
        # if (doc_key, chunk_idx) in processed:
        #     continue
        # else:
        #     processed.add((doc_key, chunk_idx))
        
        # if int(chunk_index) in data_json[int(doc_key)]:
        processed_entities = data_json[int(doc_key)][chunk_idx]

        # messages.append({"role": "user", "content": get_content(text)})

        # output, time_diff = get_prediction(text)
        # output, time_diff = get_chat_prediction(messages)
        output, time_diff = get_relation_prediction(i, processed_entities)

        outputText = output.choices[0].message.content
        input_tokens = output.usage.prompt_tokens
        output_tokens = output.usage.completion_tokens

        # messages.append({"role": "assistant", "content": outputText})

        if debug:
            print(f"\nInput - {text}")
            # print(f"\nGT - {data['Triplets'][i]}")
            print(f"\noutput - {outputText}")

        output_data.append([doc_key, chunk_idx, outputText, time_diff,
                           input_tokens, output_tokens])
            
        # else:
        #     output_data.append([doc_key, chunk_idx, "No data", 0,
        #                        0, 0])

        if i % 10 == 0 and i > 1:

            with open(output_filename, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(output_data)
                
        if debug: break
        
    if debug: break
    
    with open(output_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_data)

100%|██████████| 847/847 [00:06<00:00, 128.34it/s]


In [21]:
output_data[-1]

['400',
 0,
 'News Online Today | publication date | 2011/05/15\nABILA | country | Kronos\nProtectors of Kronos | country | Kronos\nDRUG RELATED ARRESTS ON THE RISE | publication date | 2011/05/15\nDRUG RELATED ARRESTS ON THE RISE | publisher | News Online Today\nProtectors of Kronos | located in the administrative territorial entity | Kronos\nABILA | located in the administrative territorial entity | Kronos\nNews Online Today | located in the administrative territorial entity | ABILA\nNews Online Today | country | Kronos\nDRUG RELATED ARRESTS ON THE RISE | located in the administrative territorial entity | ABILA\nDRUG RELATED ARRESTS ON THE RISE | country | Kronos',
 6.3870508670806885,
 1125,
 161]

In [19]:
data['Documents']['399']

[{'chunk_index': 0,
  'chunk_text': "SOURCE: News Online Today\n\nTITLE: KRONOS GOVERNMENT PLANS TAX BREAKS TO ENCOURAGE FOREIGN INVESTMENT \n\nPUBLISHED: 1992/12/12\n \nLOCATION: ABILA, Kronos \n\nThe Government of Kronos is currently planning some of the world's most generous tax breaks for gas investments.\n\nThe Government of Kronos is pitching the plan as a win-win situation given that the high-income countries will be struggling to discover and exploit ever-dwindling supplies of fossil-fuels and the developing Kronos needs income, infrastructure, and cheap energy.",
  'chunk_tokens': 85}]

In [48]:
int(0)

0

In [29]:
processed.remove(('3',0))

In [44]:
doc_key, chunk_index, data_json[3][0]

('3',
 1,
 'All News Today; 2012/04/09; ABILA; Kronos; Silvia Marek; Protectors of Kronos')

## Process Entities

In [ ]:
Data0 = pd.read_csv("openai_mc1_entities_Apr13.csv", header=None)
Data0 = Data0.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
                            4: 'input_tokens',  5: 'output_tokens'})
Data0["processed_entities"] = ""
print(Data0.shape)

Data0 = Data0.iloc[9:]
Data0 = Data0.reset_index(drop=True)
print(Data0.shape)

Data0.head()

In [ ]:
import re

for i in range(Data0.shape[0]):
    
    # if i != 261: # 1, 5, 27
    #     continue
        
    ent = Data0["entities"][i]
    ent2 = ent.split("; ")
    
    ent3 = []
    for item in ent2:
        item = item.strip()
        if item and item not in ent3:
            ent3.append(item)
    ent3 = "; ".join(ent3)
    
#     if ent != ent3:

#         print(f"\n{i}")
#         print(ent)
#         print(f"\n{ent3}")
#         print("*"*20)
    
    Data0["processed_entities"][i] = ent3

In [ ]:
# Data0.head()
Data0.to_csv('openai_mc1_entities_Apr13_processed.csv', index=False)

## Combine results

In [3]:
homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai"

main_output = 'openai_mc1_triplets_combined.csv'

DataPath0 = os.path.join(homeFolder, "openai_mc1_triplets_Apr13.csv")
DataPath1 = os.path.join(homeFolder, "openai_mc1_triplets_Apr13_400.csv")
# DataPath2 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_v5.csv")
# DataPath3 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_v6.csv")

Data0 = pd.read_csv(DataPath0, header=None)
Data1 = pd.read_csv(DataPath1, header=None)
# Data2 = pd.read_csv(DataPath2, header=None)
# Data3 = pd.read_csv(DataPath3, header=None)
Data = pd.concat([Data0, Data1], axis=0)

Data = Data.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'triplets', 3: 'latency',
                           4: 'input_tokens',  5: 'output_tokens', 6: 'article_name'})

Data.to_csv(main_output, index=False)

# print(Data.shape, len(list(set(list(Data["original_index"])))))
Data.head()

,doc_key,chunk_index,triplets,latency,input_tokens,output_tokens
0,0,0,Kyrla Halford | education | University of Abil...,18.778480,1194,498
1,0,1,Esme Nesmith | author | Master’s Thesis\nElian...,4.921607,1129,128
2,0,2,Protectors of Kronos | publication date | 1997...,8.510802,1123,214
3,0,3,Elodis Community Health Center | inception | 1...,5.819815,1111,145
4,0,4,Initial Stage | participant | Grassroots Effor...,11.607945,1161,313


In [ ]:
len(list(set(list(Data["original_index"]))))

In [ ]:
check_id = list(set(list(Data["original_index"])))

In [ ]:
check_id

## Analyse output

In [ ]:
homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai"

DataPath0 = 'openai_relations_gt_redocred_dev_all.csv'

Data0 = pd.read_csv(DataPath0)

Data0.head()

In [ ]:
Data0['entities'][4]